In [1]:
import numpy as np
import mne
import pandas as pd
import glob
import os

In [ ]:
for files in glob.glob('physionet.org/files/capslpdb/1.0.0/*.edf'):
    
    filename = os.path.basename(files)[:-4]
    if(os.path.isfile(filename+'.csv')):
        print(filename + " already done, skipping");
        continue;
        
    print("Processing\t", filename,"\n");
    edf = mne.io.read_raw_edf(files)
    header = ','.join(edf.ch_names)
    np.savetxt("temp.csv", edf.get_data().T, delimiter=',', header=header)
    data_ecg = pd.read_csv("temp.csv")
    try:
        data_ecg = data_ecg['ECG1-ECG2']
    except KeyError:
        try:
            data_ecg = data_ecg['ECG']
        except KeyError:
            data_ecg = data_ecg['EKG']
    data_ecg.to_csv(filename+'.csv')
    print("Saved\t", filename, "\n");

Processing	 nfle39 

Extracting EDF parameters from /home/mishra.g/spring2022/hci/project/data/physionet.org/files/capslpdb/1.0.0/nfle39.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [23]:
sr_file

,Sampling Frequency(Hz)
Name,
brux1.txt,512
brux2.txt,256
ins1.txt,256
ins2.txt,512
ins3.txt,256
...,...
rbd22.txt,512
sdb1.txt,256
sdb2.txt,256


In [27]:
sr_file.loc['brux1.txt','Sampling Frequency(Hz)']

512

In [25]:
sr_file = pd.read_csv('samp_freq.csv', index_col=0)
for file in glob.glob('physionet.org/files/capslpdb/1.0.0/*.txt'):
    info_file = pd.read_csv(file, skiprows = 20, sep='\t');
    filename = os.path.basename(file);
    filename_without_ext = filename[:-4];
    if(os.path.isfile('wake_ecg/'+filename_without_ext+'.csv')):
        print(filename_without_ext + " already done, skipping");
        continue;
    sr = sr_file.loc[filename,'Sampling Frequency(Hz)'];
    print(f"Extracting wake ECG data for {filename_without_ext}");

    for i in range(len(info_file)):
        if(i==0):
            info_file.loc[i,'Data Start'] = 0;
            info_file.loc[i,'Data End'] = info_file.loc[i,'Duration[s]']*sr;
        else:
            info_file.loc[i,'Data Start'] = info_file.loc[i-1,'Data End']+1;
            info_file.loc[i,'Data End'] = info_file.loc[i,'Data Start'] + info_file.loc[i,'Duration[s]']*sr-1;
    wake_data = info_file[info_file['Sleep Stage']=='W'];
    wake_data.reset_index(inplace=True, drop=True)
    #for i in range(len(wake_data)):
    #    start
    all_ecg = pd.read_csv(filename_without_ext+'.csv');
    wake_ecg = []
    for j in range(len(wake_data)):
        #print(j)
        wake_ecg.extend(all_ecg.iloc[int(wake_data.loc[j,'Data Start']):int(wake_data.loc[j,'Data End']),1])
    wake_ecg = pd.DataFrame(wake_ecg);
    wake_ecg.to_csv('wake_ecg/'+filename_without_ext+'.csv', index=False);            

In [33]:
wake_data = info_file[info_file['Sleep Stage']=='W'];
wake_data.reset_index(inplace=True, drop=True)

'''starts = []
ends = []

for i in range(len(wake_data)):
    if i==0:
        starts.append(wake_data.loc[i,'Data Start']);
    else:
        if(wake_data.loc[i,'Data Start']==wake_data.loc[i-1,'Data End']):
            continue;
        else:
            starts.append(wake_data.loc[i,'Data Start'])
            ends.append(wake_data.loc[i+1,'Data Start']);
'''            